# match Chris's catalogs to CAMELS

In [1]:
import os
import h5py 
import numpy as np

In [2]:
_dir = '/tigress/chhahn/goleta/'
lovell = h5py.File(os.path.join(_dir, 'tng_lovell.hdf5'), 'r')

In [3]:
tng = h5py.File(os.path.join(_dir, 'tng.snap33.subfind.galaxies.LHC.hdf5'), 'r')

In [4]:
photo = [] 
for i_lhc in range(1000): 
    isubs = tng['props'][-1,:][tng['props'][-2,:].astype(int) == i_lhc].astype(int)
    
    im = np.in1d(lovell['%i/SubhaloIndex' % i_lhc][...], isubs)
    assert np.sum(isubs-lovell['%i/SubhaloIndex' % i_lhc][...][im]) == 0
    
    _photo = np.zeros((np.sum(tng['props'][-2,:].astype(int) == i_lhc), 5))
    for i_b, band in enumerate(['u', 'g', 'r', 'i', 'z']): 
        # convert Lnu to abs mag units
        Lnu = lovell['%i/attenuated/SLOAN/SDSS.%s' % (i_lhc, band)][...][im]        
        geo = 4 * np.pi * (10 * 3.08567758128e18) ** 2

        _photo[:,i_b] = -2.5 * np.log10(Lnu / geo) - 48.6
    photo.append(_photo)

In [5]:
photo = np.concatenate(photo, axis=0)

In [6]:
tng['props'][-5:-2].T

array([[-24.05015182, -24.37333679, -24.61043739],
       [-21.64406013, -21.87215042, -22.04977226],
       [-20.69135094, -21.04071236, -21.28896141],
       ...,
       [-16.58961487, -16.86730766, -17.0278244 ],
       [-15.97955227, -16.27163506, -16.43307686],
       [-15.98023033, -16.28874588, -16.46620369]])

In [7]:
photo[:,1:4]

array([[-22.91294886, -23.47860227, -23.65429341],
       [-21.01932214, -21.57093406, -21.3778677 ],
       [-19.6086894 , -20.11191313, -20.34976113],
       ...,
       [-15.49387968, -15.8711375 , -16.07878626],
       [-15.60957446, -15.95460583, -16.15745627],
       [-15.72402075, -16.01658103, -16.20353453]])

In [8]:
np.savetxt(os.path.join(_dir, 'tng_lovell.matched.sdss_ugriz.dat'), photo, header='SDSS u, g, r, i, z abs. mag.')